In [126]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [176]:
def calc_prob(data, room_num, sensor):
    room = data[['time']]

    # shift values down to get X_t-1
    room['X_t-1'] = data[room_num].shift(1)
    room['X_t'] = data[room_num]
    room['E_t'] = data[sensor]

    # drop first row (X_1)
    room.drop(room.index[0], inplace=True)
    room['X_t-1'] = room['X_t-1'].astype('int64')

    # chance values in X_t to 1 if > 1
    # motion to 1, no motion to 0
    room.loc[room['X_t'] > 1, 'X_t'] = 1
    room.loc[room['X_t-1'] > 1, 'X_t-1'] = 1
    room.loc[room['E_t'] == 'no motion', 'E_t'] = 0
    room.loc[room['E_t'] == 'motion', 'E_t'] = 1

    # calculate transition probabilities P(X_t | X_t-1)
    on = room[room['X_t-1'] == 1].drop(['time','E_t'], axis=1)
    print("Transition probabilities:")
    print("P(off | on) =", np.round(on['X_t'].value_counts()[0] / len(on), 4))
    print("P(on | on) =", np.round(on['X_t'].value_counts()[1] / len(on), 4))

    off = room[room['X_t-1'] == 0].drop(['time','E_t'], axis=1)
    print("P(off | off) =", np.round(off['X_t'].value_counts()[0] / len(off), 4))
    print("P(on | off) =", np.round(off['X_t'].value_counts()[1] / len(off), 4))

    # calculate emission probabilities P(E_t | X_t)
    on = room[room['X_t'] == 1].drop(['time','X_t-1'], axis=1)
    print("\nEmission probabilities:")
    print("P(no motion | on) =", np.round(on['E_t'].value_counts()[0] / len(on), 4))
    print("P(motion | on) =", np.round(on['E_t'].value_counts()[1] / len(on), 4))

    off = room[room['X_t'] == 0].drop(['time','X_t-1'], axis=1)
    print("P(no motion | off) =", np.round(off['E_t'].value_counts()[0] / len(off), 4))
    print("P(motion | off) =", np.round(off['E_t'].value_counts()[1] / len(off), 4))

In [643]:
# read data
data = pd.read_csv('data.csv')
print("room 16:")
calc_prob(data, "r16", "reliable_sensor1")
print("\nroom 5:")
calc_prob(data, "r5", "reliable_sensor2")
print("\nroom 25:")
calc_prob(data, "r25", "reliable_sensor3")
print("\nroom 31:")
calc_prob(data, "r31", "reliable_sensor4")
print("\nroom 1:")
calc_prob(data, "r1", "unreliable_sensor3")
print("\nroom 24:")
calc_prob(data, "r24", "unreliable_sensor4")

room 16:
Transition probabilities:
P(off | on) = 0.0126
P(on | on) = 0.9874
P(off | off) = 0.9876
P(on | off) = 0.0124

Emission probabilities:
P(no motion | on) = 0.016
P(motion | on) = 0.984
P(no motion | off) = 0.9735
P(motion | off) = 0.0265

room 5:
Transition probabilities:
P(off | on) = 0.3837
P(on | on) = 0.6163
P(off | off) = 0.9386
P(on | off) = 0.0614

Emission probabilities:
P(no motion | on) = 0.0423
P(motion | on) = 0.9577
P(no motion | off) = 0.9633
P(motion | off) = 0.0367

room 25:
Transition probabilities:
P(off | on) = 0.1307
P(on | on) = 0.8693
P(off | off) = 0.9274
P(on | off) = 0.0726

Emission probabilities:
P(no motion | on) = 0.0432
P(motion | on) = 0.9568
P(no motion | off) = 0.9663
P(motion | off) = 0.0337

room 31:
Transition probabilities:
P(off | on) = 0.0135
P(on | on) = 0.9865
P(off | off) = 0.9868
P(on | off) = 0.0132

Emission probabilities:
P(no motion | on) = 0.0194
P(motion | on) = 0.9806
P(no motion | off) = 0.9629
P(motion | off) = 0.0371

room 1:

In [132]:
# CODE TAKEN FROM WEEK 5 TUTORIAL

def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))

def prob(factor, *entry):
    """
    argument 
    `factor`, a dictionary of domain and probability values,
    `entry`, a list of values, one for each variable in the same order as specified in the factor domain.
    
    Returns p(entry)
    """

    return factor['table'][entry]       

def join(f1, f2, outcomeSpace):
    """
    argument 
    `f1`, first factor to be joined.
    `f2`, second factor to be joined.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor with a join of f1 and f2
    """
    
    # First, we need to determine the domain of the new factor. It will be union of the domain in f1 and f2
    # But it is important to eliminate the repetitions
    common_vars = list(f1['dom']) + list(set(f2['dom']) - set(f1['dom']))
    
    # We will build a table from scratch, starting with an empty list. Later on, we will transform the list into a odict
    table = list()
    
    # Here is where the magic happens. The product iterator will generate all combinations of varible values 
    # as specified in outcomeSpace. Therefore, it will naturally respect observed values
    for entries in product(*[outcomeSpace[node] for node in common_vars]):
        
        # We need to map the entries to the domain of the factors f1 and f2
        entryDict = dict(zip(common_vars, entries))
        f1_entry = (entryDict[var] for var in f1['dom'])
        f2_entry = (entryDict[var] for var in f2['dom'])
        
        # Use the fuction prob to calculate the probability 
        p1 = prob(f1, *f1_entry)           
        p2 = prob(f2, *f2_entry)           
        
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, p1 * p2))
    return {'dom': tuple(common_vars), 'table': odict(table)}


def marginalize(f, var, outcomeSpace):
    """
    argument 
    `f`, factor to be marginalized.
    `var`, variable to be summed out.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor f' with dom(f') = dom(f) - {var}
    """    
    
    # Let's make a copy of f domain and convert it to a list. We need a list to be able to modify its elements
    new_dom = list(f['dom'])
    
    new_dom.remove(var)       # Remove var from the list new_dom 
    table = list()            # Create an empty list for table
    for entries in product(*[outcomeSpace[node] for node in new_dom]):
        s = 0;                     # Initialize the summation variable s. 
        # We need to iterate over all possible outcomes of the variable var
        for val in outcomeSpace[var]:
            # To modify the tuple entries, we will need to convert it to a list
            entriesList = list(entries)
            # We need to insert the value of var in the right position in entriesList
            entriesList.insert(f['dom'].index(var), val)
            # Calculate the probability of factor f for entriesList. 
            p = prob(f, *tuple(entriesList))   
            # Sum over all values of var by accumulating the sum in s.  
            s = s + p                            
            
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, s))
    return {'dom': tuple(new_dom), 'table': odict(table)}

def evidence(var, e, outcomeSpace):
    """
    argument 
    `var`, a valid variable identifier.
    `e`, the observed value for var.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns dictionary with a copy of outcomeSpace with var = e
    """    
    # Make a copy of outcomeSpace
    newOutcomeSpace = outcomeSpace.copy()      
    # Replace the domain of variable var with a tuple with a single element e
    newOutcomeSpace[var] = (e,)                
    return newOutcomeSpace

def normalize(f):
    """
    argument 
    `f`, factor to be normalized.
    
    Returns a new factor f' as a copy of f with entries that sum up to 1
    """ 
    table = list()
    sum = 0
    for k, p in f['table'].items():
        sum = sum + p
    for k, p in f['table'].items():
        table.append((k, p/sum))
    return {'dom': f['dom'], 'table': odict(table)}

def maximize(f, var, outcomeSpace):
    """
    argument 
    `f`, factor to be marginalized.
    `var`, variable to be maximized out.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor f' with dom(f') = dom(f) - {var}
    """    
    # Let's make a copy of f domain and convert it to a list. We need a list to be able to modify its elements
    new_dom = list(f['dom'])
    new_dom.remove(var)            # Remove var from the list new_dom
    table = list()                 # Create an empty list for table.
    for entries in product(*[outcomeSpace[node] for node in new_dom]):     
        m = 0;                  # Initialize the maximization variable m.

        # We need to iterate over all possible outcomes of the variable var
        for val in outcomeSpace[var]:
            # To modify the tuple entries, we will need to convert it to a list
            entriesList = list(entries)
            # We need to insert the value of var in the right position in entriesList
            entriesList.insert(f['dom'].index(var), val)
            # Calculate the probability of factor f for entriesList.
            p = prob(f, *tuple(entriesList))     
            # Maximize over all values of var by storing the max value in m.
            m = max(m, p)                        
            
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, m))
    return {'dom': tuple(new_dom), 'table': odict(table)}

In [161]:
# CODE TAKEN FROM WEEK 5 TUTORIAL

def viterbiOnline(f, transition, emission, stateVar, emissionVar, emissionEvi, outcomeSpace, norm):
    """
    argument 
    `f`, factor that represents the previous state.
    `transition`, transition probabilities from time t-1 to t.
    `emission`, emission probabilities.
    `stateVar`, state (hidden) variable.
    `emissionVar`, emission variable.
    `emissionEvi`, emission observed evidence. If undef, we do only the time update
    `outcomeSpace`, dictionary with the domain of each variable.
    
    Returns a new factor that represents the current state.
    """ 

    # Set fCurrent as a copy of f
    fCurrent = f.copy()
    # Set the f_previous domain to be a list with a single variable name appended with '_t-1' to indicate previous time step
    fCurrent['dom'] = (stateVar + '_t-1', )       
    # Make the join operation between fCurrent and the transition probability table    
    fCurrent = join(fCurrent, transition, outcomeSpace)        
    # Eliminate the randVariable_t-1 with the maximization operation
    fCurrent = maximize(fCurrent, fCurrent['dom'][0], outcomeSpace)        
    # If emissionEvi == None, we will assume this time step has no observed evidence    
    if emissionEvi != None:                  # WARNING: do not change this line
        # Set evidence in the form emissionVar = emissionEvi    
        newOutcomeSpace = evidence(emissionVar, emissionEvi, outcomeSpace)     
        # Make the join operation between fCurrent and the emission probability table. Use the newOutcomeSpace    
        fCurrent = join(fCurrent, emission, newOutcomeSpace)      
        # Marginalize emissionVar. Use the newOutcomeSpace    
        fCurrent = marginalize(fCurrent, emissionVar, newOutcomeSpace)         
        # Normalize fcurrent.
        if norm:
            fCurrent = normalize(fCurrent)           
    # Set the domain of w to be name of the random variable without time index
    fCurrent['dom'] = (stateVar, )
    return fCurrent

In [661]:
outcomeSpaceLights = {
    "Lights_t": ('on','off'),
    "Lights_t-1": ('on','off'),
}

startLights = {
    'dom': ('Lights',), 
    'table': odict([
        (('on',), 0.0),
        (('off',), 1.0),
    ])
}

fCurrent16 = viterbiOnline(startLights, transitionLights16, evidenceLights16, 'Lights', 'Sensor_t', None, outcomeSpaceLights, norm=True)

fCurrent5 = viterbiOnline(startLights, transitionLights5, evidenceLights5, 'Lights', 'Sensor_t', None, outcomeSpaceLights, norm=True)

fCurrent25 = viterbiOnline(startLights, transitionLights25, evidenceLights25, 'Lights', 'Sensor_t', None, outcomeSpaceLights, norm=True)

fCurrent31 = viterbiOnline(startLights, transitionLights31, evidenceLights31, 'Lights', 'Sensor_t', None, outcomeSpaceLights, norm=True)

fCurrent1 = viterbiOnline(startLights, transitionLights1, evidenceLights1, 'Lights', 'Sensor_t', None, outcomeSpaceLights, norm=True)

fCurrent24 = viterbiOnline(startLights, transitionLights24, evidenceLights24, 'Lights', 'Sensor_t', None, outcomeSpaceLights, norm=True)

In [654]:
printFactor(fCurrent16)

| Lights   |     Pr |
|----------+--------|
| on       | 0.0124 |
| off      | 0.9876 |


In [648]:
# room 16, reliable sensor 1
transitionLights16 = {
    'dom': ('Lights_t-1', 'Lights_t'), 
    'table': odict([
        (('on','on'), 0.9874),
        (('on','off'), 0.0126),
        (('off','on'), 0.0124),
        (('off','off'), 0.9876),        
    ])
}

evidenceLights16 = {
    'dom': ('Lights_t', 'Sensor_t'), 
    'table': odict([
        (('on','motion'), 0.984),
        (('on','no motion'), 0.016),
        (('off','motion'), 0.0265),
        (('off','no motion'), 0.9735),
    ])
}


emissionEvi = 'motion'
fCurrent16 = viterbiOnline(fCurrent16, transitionLights16, evidenceLights16, 'Lights', 'Sensor_t', emissionEvi, outcomeSpaceLights, norm=True)
printFactor(fCurrent16)
print(max(fCurrent16['table'], key=fCurrent16['table'].get)[0])

| Lights   |       Pr |
|----------+----------|
| on       | 0.317974 |
| off      | 0.682026 |
off


In [ ]:
# room 5, reliable sensor 2
transitionLights5 = {
    'dom': ('Lights_t-1', 'Lights_t'), 
    'table': odict([
        (('on','on'), 0.6163),
        (('on','off'), 0.3837),
        (('off','on'), 0.0614),
        (('off','off'), 0.9386),        
    ])
}

evidenceLights5 = {
    'dom': ('Lights_t', 'Sensor_t'), 
    'table': odict([
        (('on','motion'), 0.9577),
        (('on','no motion'), 0.0423),
        (('off','motion'), 0.0367),
        (('off','no motion'), 0.9633),
    ])
}

emissionEvi = 'motion'
fCurrent5 = viterbiOnline(fCurrent5, transitionLights5, evidenceLights5, 'Lights', 'Sensor_t', emissionEvi, outcomeSpaceLights, norm=True)
printFactor(fCurrent5)
print(max(fCurrent5['table'], key=fCurrent5['table'].get)[0])

In [ ]:
# room 25, reliable sensor 3
transitionLights25 = {
    'dom': ('Lights_t-1', 'Lights_t'), 
    'table': odict([
        (('on','on'), 0.8693),
        (('on','off'), 0.1307),
        (('off','on'), 0.0726),
        (('off','off'), 0.9274),        
    ])
}

evidenceLights25 = {
    'dom': ('Lights_t', 'Sensor_t'), 
    'table': odict([
        (('on','motion'), 0.9568),
        (('on','no motion'), 0.0432),
        (('off','motion'), 0.0337),
        (('off','no motion'), 0.9663),
    ])
}

emissionEvi = 'motion'
fCurrent25 = viterbiOnline(fCurrent25, transitionLights25, evidenceLights25, 'Lights', 'Sensor_t', emissionEvi, outcomeSpaceLights, norm=True)
printFactor(fCurrent25)
print(max(fCurrent25['table'], key=fCurrent25['table'].get)[0])

In [ ]:
# room 31, reliable sensor 4
transitionLights31 = {
    'dom': ('Lights_t-1', 'Lights_t'), 
    'table': odict([
        (('on','on'), 0.9865),
        (('on','off'), 0.0135),
        (('off','on'), 0.01323),
        (('off','off'), 0.9868),        
    ])
}

evidenceLights31 = {
    'dom': ('Lights_t', 'Sensor_t'), 
    'table': odict([
        (('on','motion'), 0.9806),
        (('on','no motion'), 0.0194),
        (('off','motion'), 0.0371),
        (('off','no motion'), 0.9629),
    ])
}

emissionEvi = 'motion'
fCurrent31 = viterbiOnline(fCurrent31, transitionLights31, evidenceLights31, 'Lights', 'Sensor_t', emissionEvi, outcomeSpaceLights, norm=True)
printFactor(fCurrent31)
print(max(fCurrent31['table'], key=fCurrent31['table'].get)[0])

In [659]:
# room 1, unreliable sensor 3
transitionLights1 = {
    'dom': ('Lights_t-1', 'Lights_t'), 
    'table': odict([
        (('on','on'), 0.9434),
        (('on','off'), 0.0566),
        (('off','on'), 0.0741),
        (('off','off'), 0.9259),        
    ])
}

evidenceLights1 = {
    'dom': ('Lights_t', 'Sensor_t'), 
    'table': odict([
        (('on','motion'), 0.9515),
        (('on','no motion'), 0.0485),
        (('off','motion'), 0.1405),
        (('off','no motion'), 0.8595),
    ])
}

emissionEvi = 'motion'
fCurrent1 = viterbiOnline(fCurrent1, transitionLights1, evidenceLights1, 'Lights', 'Sensor_t', emissionEvi, outcomeSpaceLights, norm=True)
printFactor(fCurrent1)
print(max(fCurrent1['table'], key=fCurrent1['table'].get)[0])

| Lights   |       Pr |
|----------+----------|
| on       | 0.355765 |
| off      | 0.644235 |
off


In [665]:
# room 24, unreliable sensor 4
transitionLights24 = {
    'dom': ('Lights_t-1', 'Lights_t'), 
    'table': odict([
        (('on','on'), 0.5378),
        (('on','off'), 0.4622),
        (('off','on'), 0.0281),
        (('off','off'), 0.9719),        
    ])
}

evidenceLights24 = {
    'dom': ('Lights_t', 'Sensor_t'), 
    'table': odict([
        (('on','motion'), 0.9328),
        (('on','no motion'), 0.0672),
        (('off','motion'), 0.1438),
        (('off','no motion'), 0.8562),
    ])
}

emissionEvi = 'motion'
fCurrent24 = viterbiOnline(fCurrent24, transitionLights24, evidenceLights24, 'Lights', 'Sensor_t', emissionEvi, outcomeSpaceLights, norm=True)
printFactor(fCurrent24)
print(max(fCurrent24['table'], key=fCurrent24['table'].get)[0])

| Lights   |       Pr |
|----------+----------|
| on       | 0.883011 |
| off      | 0.116989 |
on


In [642]:
# calculate failure rate of unreliable sensor
def calc_failure_rate(data, sensor, room_num):
    room = data[['time']]
    room[sensor] = data[sensor]
    room[room_num] = data[room_num]

    # chance values in X_t to 1 if > 1
    # motion to 1, no motion to 0
    room.loc[room[room_num] > 1, room_num] = 1
    room.loc[room[sensor] == 'no motion', sensor] = 0
    room.loc[room[sensor] == 'motion', sensor] = 1

    failed = len(room[(room[sensor] == 0) & (room[room_num] == 1)]) + len(room[(room[sensor] == 1) & (room[room_num] == 0)])
    return failed / len(room)

# print(calc_failure_rate(data, 'unreliable_sensor1', 'o1'))
# print(calc_failure_rate(data, 'unreliable_sensor2', 'c3'))
print(calc_failure_rate(data, 'unreliable_sensor3', 'r1'))
print(calc_failure_rate(data, 'unreliable_sensor4', 'r24'))

0.08829654310703873
0.13994169096209913


In [383]:
map = {
    'outside': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r12']
    },
    'r1': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r2','r3']
    },
    'r2': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r1','r4']
    },
    'r3': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r1','r7']
    },
    'r4': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r2','r8']
    },
    'r5': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r6','r9','c3']
    },
    'r6': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r5','c3'] 
    },
    'r7': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r3','c1']
    },
    'r8': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r4','r9']
    },
    'r9': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r5','r8','r13']
    },
    'r10': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r11': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r12': {
        'ppl_in_room': [],
        'adjacent_rooms': ['outside','r22']
    },
    'r13': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r9','r24']
    },
    'r14': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r24']
    },
    'r15': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r16': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r17': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r18': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r19': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r20': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r21': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3']
    },
    'r22': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r12','r25']
    },
    'r23': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r24']
    },
    'r24': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r13','r14','r23']
    },
    'r25': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r22','r26','c1']
    },
    'r26': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r25','r27']
    },
    'r27': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r26','r32']
    },
    'r28': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c4']
    },
    'r29': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r30','c4']
    },
    'r30': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r29']
    },
    'r31': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r32']
    },
    'r32': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r27','r31','r33']
    },
    'r33': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r32']
    },
    'r34': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c2']
    },
    'r35': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c4']
    },
    'c1': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r7','r25','c2']
    },
    'c2': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r34','c1','c4']
    },
    'c3': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r5','r6','r10','r11','r15','r16','r17','r18','r19','r20','r21','o1']
    },
    'c4': {
        'ppl_in_room': [],
        'adjacent_rooms': ['r28','r29','r35','c2','o1']
    },
    'o1': {
        'ppl_in_room': [],
        'adjacent_rooms': ['c3','c4']
    }
}

In [499]:
# TODO: NEED TO FIGURE OUT PROPER TRANSITION PROBABILITIES
# FOR NOW USE UNIFORM DISTRIBUTION

def get_transition_model(map):
    transitionModel = {}
    for room, neighbours in map.items():
        states = {}
        states[room] = np.nan
        # add neighbouring rooms to possible next state
        for neighbour in neighbours['adjacent_rooms']:
            # avoid duplicates
            if neighbour not in states:
                states[neighbour] = np.nan
            # add rooms next to neighbouring rooms to possible next state
            for next_neighbour in map[neighbour]['adjacent_rooms']:
                # avoid ownself and duplicates
                if next_neighbour != room and next_neighbour not in states:
                    states[next_neighbour] = np.nan
        # set uniform distribution for transition probabilities
        for possibleStates in states:
            states[possibleStates] = 1 / len(states)

        transitionModel[room] = states

    return transitionModel

In [490]:
# calculates cumulative distribution function
def get_cdf(pdf):
    cdf = {}
    sum = 0
    for room, prob in pdf.items():
        sum += prob
        cdf[room] = np.round(sum,4)

    return cdf

# resamples new room for the people in a given room
def get_new_room(transitionProb):
    sampled_prob = np.random.uniform(0,1)
    transitionCdf = get_cdf(transitionProb)

    # makes sure cdf is in sorted order
    for room, cdf in sorted(transitionCdf.items(), key=lambda x: x[1]):
        if sampled_prob <= cdf:
            return room

In [500]:
# sample number of workers from Normal distribution
num_ppl = np.int32(np.round(np.random.normal(20, 1)))
print(num_ppl)

# assign everyone to outside at the start (08:00:00)
new_map = copy.deepcopy(map)
for i in range(num_ppl):
    new_map['outside']['ppl_in_room'].append(i)

20


In [524]:
def simulate(map):
    transitionModel = get_transition_model(map)
    # new dict to store updated rooms of each person
    new_rooms = {}
    for room in map:
        new_rooms[room] = []

    # approximate movement by resampling transition probabilities of each room
    moved = []
    for room in map:
        # only check rooms that are not empty
        if len(map[room]['ppl_in_room']):
            # print("In:", room)
            for ppl in map[room]['ppl_in_room']:
                # print(ppl)
                # skip if already moved
                if ppl in moved:
                    print("already moved:", ppl)
                    continue
                # mark person as moved
                moved.append(ppl)
                # assign new room to each people 
                new_room = get_new_room(transitionModel[room])
                new_rooms[new_room].append(ppl)

    # update map according to newly assigned rooms
    for room in map:
        update = copy.deepcopy(new_rooms[room])
        map[room]['ppl_in_room'] = update

simulate(new_map)
new_map

{'outside': {'ppl_in_room': [], 'adjacent_rooms': ['r12']},
 'r1': {'ppl_in_room': [], 'adjacent_rooms': ['r2', 'r3']},
 'r2': {'ppl_in_room': [], 'adjacent_rooms': ['r1', 'r4']},
 'r3': {'ppl_in_room': [], 'adjacent_rooms': ['r1', 'r7']},
 'r4': {'ppl_in_room': [19], 'adjacent_rooms': ['r2', 'r8']},
 'r5': {'ppl_in_room': [], 'adjacent_rooms': ['r6', 'r9', 'c3']},
 'r6': {'ppl_in_room': [9], 'adjacent_rooms': ['r5', 'c3']},
 'r7': {'ppl_in_room': [4, 8], 'adjacent_rooms': ['r3', 'c1']},
 'r8': {'ppl_in_room': [14], 'adjacent_rooms': ['r4', 'r9']},
 'r9': {'ppl_in_room': [16], 'adjacent_rooms': ['r5', 'r8', 'r13']},
 'r10': {'ppl_in_room': [], 'adjacent_rooms': ['c3']},
 'r11': {'ppl_in_room': [], 'adjacent_rooms': ['c3']},
 'r12': {'ppl_in_room': [11], 'adjacent_rooms': ['outside', 'r22']},
 'r13': {'ppl_in_room': [], 'adjacent_rooms': ['r9', 'r24']},
 'r14': {'ppl_in_room': [], 'adjacent_rooms': ['r24']},
 'r15': {'ppl_in_room': [], 'adjacent_rooms': ['c3']},
 'r16': {'ppl_in_room': 

In [566]:
# decide whether to turn lights on or off based on particle filtering
def particle_filtering_actions(map):
    actions = {}
    for room in map:
        if room[0] == 'r':
            num = re.findall("\d+", room)[0]
            lights = 'lights' + num
            if len(map[room]['ppl_in_room']):
                actions[lights] = 'on'
            else:
                actions[lights] = 'off'

    return actions

actions = particle_filtering_actions(new_map)

In [567]:
df = data[['time','outside','r12','r22','r25','r26','r27','r32','r31','r33','c1','c2','r7']]
df.set_index('time').head(15)

,outside,r12,r22,r25,r26,r27,r32,r31,r33,c1,c2,r7
time,,,,,,,,,,,,
08:00:15,17,5,1,0,0,0,0,0,0,0,0,0
08:00:30,15,4,3,0,0,0,0,0,0,1,0,0
08:00:45,0,10,8,4,0,0,0,0,0,1,0,0
08:01:00,0,0,10,8,0,0,0,0,0,5,0,0
08:01:15,0,0,2,8,0,0,0,0,0,11,2,0
08:01:30,0,0,0,3,2,0,0,0,0,12,6,0
08:01:45,0,0,0,0,0,3,0,0,0,8,12,0
08:02:00,0,0,0,0,0,3,0,0,0,2,13,0
08:02:15,0,0,0,0,0,1,2,0,0,1,10,1
